In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from datetime import date, datetime, timedelta, time
import pandas as pd
import seaborn
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import IPython.html.widgets as widgets

In [ ]:
import scipy
from scipy.stats import norm
import scipy.optimize as op
import emcee
import triangle

In [ ]:
seaborn.set_context('talk')

In [ ]:
seaborn.set_context('poster')
bigfontsize=40
labelfontsize=35
tickfontsize=25
# seaborn.set_context('talk')
# bigfontsize=30
# labelfontsize=25
# tickfontsize=20

plt.rcParams.update({'font.size': bigfontsize})

In [ ]:
import simtransient as st
import simtransient.utils as stutils
from simtransient import hammer
from simtransient.models.supernova import Sn1aOpticalEnsemble, Sn2OpticalEnsemble

In [ ]:
Sn2OpticalEnsemble

In [ ]:
ens = Sn1aOpticalEnsemble()
# ens = Sn2OpticalEnsemble()

In [ ]:
np.linalg.det(ens.gauss_cov)

In [ ]:
ens.gauss_pars

In [ ]:
print np.linalg.det(ens.gauss_cov), np.prod(ens.gauss_pars.T.mu)
print np.linalg.det(ens.gauss_cov)/np.prod(ens.gauss_pars.T.mu),

In [ ]:
ens.gauss_correlations

In [ ]:
# from scipy.stats import multivariate_normal
# gauss_hypers_rv = multivariate_normal(mean=ens.gauss_pars.loc['mu'], cov=ens.gauss_cov)

In [ ]:
sampler, init_ball = hammer.prep_ensemble_sampler(ens.gauss_pars.T.mu.values, 
                                       ens.gauss_lnprior, 
                                       args=[],
                                       nwalkers=100,
                                       ballsize=1e-4,
                                       threads=4)
_ = sampler.run_mcmc(init_ball, N=800)

In [ ]:
cs, sample = st.hammer.trim_chain(sampler, pt=False)

In [ ]:
st.plot.chain.all_walkers(sampler.chain, cs, ens.gauss_pars.keys())

In [ ]:
# true_params = pd.Series(index=ens.gauss_pars.keys(), 
#                         data=gauss_hypers_rv.rvs(), 
#                         name="true_params")
true_params = pd.Series(index=ens.gauss_pars.keys(), 
                data=sample[np.random.choice(len(sample))],
                         name="true_params")
true_params['t0']=0
true_params

In [ ]:
true_curve = ens.get_curve(**true_params)
# true_curve

In [ ]:
# n_samples=5000
# sample = gauss_hypers_rv.rvs(n_samples)


seaborn.set_context('poster')
seaborn.set_style('dark')
_=triangle.corner(sample,
                 labels=ens.gauss_pars.keys(),
#                  quantiles=[0.05, 0.5, 0.95],
#                  truths=true_params
                 )
plt.savefig('multivar.pdf')

In [ ]:
true_params

In [ ]:
sample[0]

In [ ]:

def plt_traces(ntrace):
#     ntrace=int(ntrace)
    
    subsamples = sample[np.random.choice(len(sample), size=ntrace, replace=False)]

    t=np.linspace(-20, 80,1000)
    lcs=[]
    for pltpars in subsamples:
#         print "PLTPLARS:", pltpars
        lcs.append(ens.evaluate(t,*pltpars,t0=10))
    lcs=np.array(lcs)
    seaborn.tsplot( lcs,  t, err_style="unit_traces", ls='')
    plt.xlabel('Time')
    plt.ylabel('Flux')

# widgets.interact_manual(plt_traces, 
#          ntrace=(0,150),
#         )
plt_traces(50)
plt.ylim(0,18)
# plt.savefig('lc_family.pdf')

In [ ]:
# ens = Sn1aOpticalEnsemble()
# ens = Sn2OpticalEnsemble()

In [ ]:
param_ranges = pd.DataFrame(columns = ens.gauss_pars.keys())
n_sigma = 2
param_ranges.loc['min'] = ens.gauss_pars.T.mu - n_sigma*ens.gauss_pars.T.sigma
param_ranges.loc['max'] = ens.gauss_pars.T.mu + n_sigma*ens.gauss_pars.T.sigma
param_ranges.loc['step'] = ens.gauss_pars.T.sigma

# param_ranges.b['max']=10e-3
param_ranges

In [ ]:
interact_ranges = dict(zip(param_ranges.keys(), param_ranges.values.T))
interact_ranges = {k:tuple(v) for k,v in interact_ranges.items()}
interact_ranges

In [ ]:
print ens.curve_class.__doc__

In [ ]:
seaborn.set_style('darkgrid')
tsteps=np.linspace(-100, 100,1000)

def curveplot(**kwargs):
    curve = ens.get_curve(**kwargs)
#     print curve
    flux = curve(tsteps)
    plt.plot(tsteps,flux)
    plt.ylim(0,10)
    plt.xlim(-40,100)
    plt.xlabel('Time')
    plt.ylabel('Flux')
#     plt.suptitle('A parametric model')

widgets.interact(curveplot, t0=widgets.fixed(0), **interact_ranges)
#plt.savefig('parametric.pdf')